In [199]:
import pandas as pd

DATA_PATH_RUI = './data/bq-results-20190710-040500-jn11labq5atn.json'
DATA_PATH_SIM = './data/bq-results-20190710-041233-sg10wldt6l6m.json'

def read_jsonl(path):
    with open(path, 'r') as f:
        data = f.readlines()
        
    data = map(lambda x: x.rstrip(), data)
    data_json_str = "[" + ','.join(data) + "]"
    return pd.read_json(data_json_str)

df_rui = read_jsonl(DATA_PATH_RUI).query('tournament=="NCAA"')
df_sim = read_jsonl(DATA_PATH_SIM)

print(df_rui.columns)


Index(['abbr_name', 'active', 'assists', 'assists_turnover_ratio',
       'birth_place', 'birthplace_city', 'birthplace_country',
       'birthplace_state', 'blocked_att', 'blocks', 'class', 'conf_alias',
       'conf_name', 'defensive_rebounds', 'division_alias', 'division_name',
       'field_goals_att', 'field_goals_made', 'field_goals_pct', 'first_name',
       'flagrant_fouls', 'free_throws_att', 'free_throws_made',
       'free_throws_pct', 'full_name', 'game_id', 'gametime', 'height',
       'home_team', 'jersey_number', 'last_name', 'league_name', 'minutes',
       'minutes_int64', 'neutral_site', 'offensive_rebounds', 'personal_fouls',
       'played', 'player_id', 'points', 'position', 'primary_position',
       'rebounds', 'scheduled_date', 'season', 'sp_created', 'starter',
       'status', 'steals', 'team_alias', 'team_id', 'team_market', 'team_name',
       'tech_fouls', 'three_points_att', 'three_points_made',
       'three_points_pct', 'tournament', 'tournament_game_no'

In [200]:
names = df_sim.abbr_name.unique()
print(f'The number of similar players to Rui Hachimura = {len(names)}')

The number of similar players to Rui Hachimura = 1300


In [201]:
str_labels = [
    # プレイヤーID
    'player_id',
    
    # プレイヤー名
    'full_name',
    
    # シーズン
    'season',
]

num_labels = [
    # アシスト数
    'assists',

    # ブロック数
    'blocks',
    
    # ディフェンスリバウンド数
    'defensive_rebounds',

    # フィールドゴール試投数
    'field_goals_att',
    
    # フィールドゴール成功数
    'field_goals_made',
    
    # フリースロー試投数
    'free_throws_att',
    
    # フリースロー成功数
    'free_throws_made',
    
    # 身長
    'height',
    
    # 出場時間
    'minutes',
    
     # オフェンスリバウンド数
    'offensive_rebounds',
    
    # 得点数
    'points',
    
    # リバウンド数
    'rebounds',
    
    # スティール数
    'steals',
    
    # 3P試投数
    'three_points_att',
    
    # 3P成功数
    'three_points_made',
    
    # ターンオーバー数
    'turnovers',
    
    # 2P試投数
    'two_points_att',
    
    # 2P成功数
    'two_points_made',
    
    # 体重
    'weight',
]

# 2018年のデータがないので手動で追加
# https://www.espn.com/mens-college-basketball/team/stats/_/id/2250
rui_2018 = {
    'abbr_name': ' R.Hachimura',
    'active': 1.0,
    'assists': 1.5,
    'blocked_att': None,
    'blocks': 0.7,
    'defensive_rebounds': 188/37,
    'field_goals_att': 465/37,
    'field_goals_made': 275/37,
    'field_goals_pct': 59.1,
    'first_name': "Rui",
    'flagrant_fouls': None,
    'free_throws_att': 222/37,
    'free_throws_made': 164/37,
    'free_throws_pct': 73.9,
    'full_name': "Rui Hachimura",
    'height': 80,
    'last_name': "Hachimura",
    'minutes': 30.2,
    'offensive_rebounds': 52/37,
    'personal_fouls': None,
    'player_id': '66a86e68-5bfc-4e7a-9610-1a7a7ae406b4',
    'points': 19.7,
    'position': "F",
    'primary_position': "NA",
    'rebounds': 6.5,
    'season': 2018,
    'starter': True,
    'status': "ACT",
    'steals': 0.9,
    'team_id': '2f4d21f8-6d5f-48a5-abca-52a30583871a',
    'team_market': 'Gonzaga',
    'team_name': 'Bulldogs',
    'tech_fouls': None,
    'three_points_att': 36/37,
    'three_points_made': 15/37,
    'three_points_pct': 41.7,
    'tournament': "NCAA",
    'turnovers': 1.8,
    'two_points_att': (465-36)/37,
    'two_points_made': (275-15)/37,
    'two_points_pct': 59.1,
    'weight': 225,
}
df_rui = pd.DataFrame([rui_2018])

In [202]:
def convert_minutes(val):
    if type(val) is int:
        return float(val)
    elif type(val) is float:
        return val
    elif type(val) is str:
        time = val.split(':')
        return float(time[0]) + float(time[1]) / 60.0
    else:
        return 0.0

df_rui['minutes'] = df_rui.minutes.apply(convert_minutes)
df_sim['minutes'] = df_sim.minutes.apply(convert_minutes)

In [203]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

df_dummy = df_sim.loc[:, num_labels].dropna()
knn = NearestNeighbors(n_neighbors=3, algorithm='ball_tree')
knn.fit(df_dummy)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                 radius=1.0)

In [204]:
def is_number(x):
    return (type(x) is int or type(x) is float) and not np.isnan(x)


def neighbor_or_mean(df, label, neighbors):
    mean = df[label].dropna().mean()

    def f(row):        
        v = None
        if is_number(row[label]):
             v = row[label]

        else:
            for neigh in neighbors:
                x = df.loc[neigh, [label]]
                if is_number(x):
                    v = x
                    break

            # 最近傍のデータが値を持たなければ平均値を使用する
            if v is None:
                v = mean
        return v    
    return f


def fillna(df):
    _df = df.copy()
    
#     _, neighbors = knn.kneighbors(_df.loc[:, num_labels].fillna(0.0))
    neighbors = []

    for label in num_labels:
        _df[label] = df.apply(neighbor_or_mean(df, label, neighbors), axis=1)
    return _df


_df_sim = fillna(df_sim)
df_sim = _df_sim

In [208]:
df_rui_stats = df_rui.loc[:, labels].groupby(['player_id', 'season', 'full_name']).mean()
df_sim_stats = df_sim.loc[:, labels].groupby(['player_id', 'season', 'full_name']).mean()

In [209]:
df_rui_stats.head(1)

,,,assists,blocks,defensive_rebounds,field_goals_att,field_goals_made,free_throws_att,free_throws_made,height,minutes,offensive_rebounds,points,rebounds,starter,steals,three_points_att,three_points_made,turnovers,two_points_att,two_points_made,weight
player_id,season,full_name,,,,,,,,,,,,,,,,,,,,
66a86e68-5bfc-4e7a-9610-1a7a7ae406b4,2018,Rui Hachimura,1.5,0.7,5.081081,12.567568,7.432432,6.0,4.432432,80,30.2,1.405405,19.7,6.5,True,0.9,0.972973,0.405405,1.8,11.594595,7.027027,225


In [210]:
df_sim_stats.head(3)

,,,assists,blocks,defensive_rebounds,field_goals_att,field_goals_made,free_throws_att,free_throws_made,height,minutes,offensive_rebounds,points,rebounds,starter,steals,three_points_att,three_points_made,turnovers,two_points_att,two_points_made,weight
player_id,season,full_name,,,,,,,,,,,,,,,,,,,,
0060a1bd-1c7b-43da-8690-63962bd28aa6,2015,Zach Smith,1.000000,2.00000,4.000000,6.000000,4.000000,2.000000,0.000000,80,34.000000,0.000000,8.000000,4.000000,1.0,1.000000,0.000000,0.000000,3.000000,6.000000,4.000000,220
00637b09-c75d-438a-82c7-02467546bbfb,2015,Jaylin McDonald,0.185062,0.12826,0.637154,1.227746,0.588134,0.444815,0.302608,78,17.486401,0.311306,1.568657,0.947693,0.0,0.111785,0.261876,0.090257,0.224378,0.965364,0.497425,205
00ae065b-9c78-49b8-b9cf-5d8872ef2d1d,2014,Georgi Funtarov,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,80,17.486401,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,214


In [211]:
# dataframe を結合
df = pd.concat([df_rui_stats.loc[:, num_labels], df_sim_stats.loc[:, num_labels]])

In [212]:
# 各値を標準化（N(0, 1) に従う分布に変換）する
def standardize(df, labels):
    _df = df.copy()
    for label in labels:
        mean = df[label].mean()
        std = df[label].std()
        values = []
        for val in df[label].values:
            if std > 0:
                values.append((val - mean) / std)
            else:
                values.append(0.0)
        _df[label] = values
    return _df

df = standardize(df, num_labels)

In [213]:
# 各値を [0, 1] に正規化
def normalize(df, labels):
    _df = df.copy()
    for label in labels:
        max_val = df[label].max()
        min_val = df[label].min()
        values = []
        for val in df[label].values:
            if max_val != min_val:
                values.append((val - min_val) / (max_val - min_val))
            else:
                values.append(0.0)
        _df[label] = values
    return _df

# df = normalize(df, num_labels)

In [214]:
df.head(2)

,,,assists,blocks,defensive_rebounds,field_goals_att,field_goals_made,free_throws_att,free_throws_made,height,minutes,offensive_rebounds,points,rebounds,steals,three_points_att,three_points_made,turnovers,two_points_att,two_points_made,weight
player_id,season,full_name,,,,,,,,,,,,,,,,,,,
66a86e68-5bfc-4e7a-9610-1a7a7ae406b4,2018,Rui Hachimura,1.141035,0.525911,1.523418,2.175980,2.792317,2.506178,2.588536,0.170032,1.341872,0.399980,2.828208,1.245405,1.021130,0.106052,0.211940,1.179414,2.600398,3.058718,0.065675
0060a1bd-1c7b-43da-8690-63962bd28aa6,2015,Zach Smith,0.545136,2.558968,1.009754,0.560716,1.109352,0.368754,-0.642686,0.170032,1.744487,-0.789117,0.632685,0.402863,1.201299,-0.535500,-0.438531,2.464911,0.925816,1.394809,-0.383849


In [215]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=16)
kmeans.fit(df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=16, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [216]:
df['cluster'] = kmeans.labels_

In [217]:
from sklearn.metrics.pairwise import cosine_similarity

def calc_cos(row, df):
    cosines = []
    for _, x in df.iterrows():
        X = row.values.reshape(1, -1)
        Y = x.values.reshape(1, -1)
        cosines.append(cosine_similarity(X, Y)[0][0])
    return cosines

cos = calc_cos(df.iloc[0], df)
df['cosine'] = cos

In [218]:
df.head(2)

,,,assists,blocks,defensive_rebounds,field_goals_att,field_goals_made,free_throws_att,free_throws_made,height,minutes,offensive_rebounds,...,rebounds,steals,three_points_att,three_points_made,turnovers,two_points_att,two_points_made,weight,cluster,cosine
player_id,season,full_name,,,,,,,,,,,,,,,,,,,,,
66a86e68-5bfc-4e7a-9610-1a7a7ae406b4,2018,Rui Hachimura,1.141035,0.525911,1.523418,2.175980,2.792317,2.506178,2.588536,0.170032,1.341872,0.399980,...,1.245405,1.021130,0.106052,0.211940,1.179414,2.600398,3.058718,0.065675,0,1.000000
0060a1bd-1c7b-43da-8690-63962bd28aa6,2015,Zach Smith,0.545136,2.558968,1.009754,0.560716,1.109352,0.368754,-0.642686,0.170032,1.744487,-0.789117,...,0.402863,1.201299,-0.535500,-0.438531,2.464911,0.925816,1.394809,-0.383849,6,0.365519


In [219]:
no_cluster_hachimura = df.cluster.values[0]
print(f'cluster no. of Hachimura = {no_cluster_hachimura}')

df.query(f'cluster=={no_cluster_hachimura}')

cluster no. of Hachimura = 0


,,,assists,blocks,defensive_rebounds,field_goals_att,field_goals_made,free_throws_att,free_throws_made,height,minutes,offensive_rebounds,...,rebounds,steals,three_points_att,three_points_made,turnovers,two_points_att,two_points_made,weight,cluster,cosine
player_id,season,full_name,,,,,,,,,,,,,,,,,,,,,
66a86e68-5bfc-4e7a-9610-1a7a7ae406b4,2018,Rui Hachimura,1.141035,0.525911,1.523418,2.175980,2.792317,2.506178,2.588536,0.170032,1.341872,0.399980,...,1.245405,1.021130,0.106052,0.211940,1.179414,2.600398,3.058718,0.065675,0,1.000000
0bd137c0-85f7-4395-a51b-88f707a4f8d1,2015,Laimonas Chatkevicius,0.545136,0.995078,0.772184,2.036389,3.806072,-0.432779,-0.278189,2.062737,0.579022,0.480016,...,0.739880,0.300453,-0.205813,-0.438531,-0.213208,2.572084,4.418073,1.863772,0,0.745111
11702dc5-fd09-4b71-8c54-a08af0c91dd2,2015,Jaylen Bond,-0.646661,0.995078,4.335733,2.774225,2.580290,-0.699957,-0.642686,0.170032,2.592098,2.595238,...,4.110050,-0.600392,-0.535500,-0.438531,2.464911,3.619709,3.043862,1.414247,0,0.670614
146aed9c-0443-49e0-b537-9fa2669af16c,2014,Jarell Martin,4.120527,-0.568812,3.385453,2.282335,2.089977,2.506178,2.273295,1.431835,2.380195,0.903060,...,2.761982,1.201299,0.123873,-0.438531,0.322416,2.721744,2.494177,0.964723,0,0.863545
1df2ebf8-f260-44e1-b41f-10739d1f02e5,2015,Ethan Happ,1.339667,0.995078,1.801653,1.544498,2.253415,1.793703,1.544299,1.431835,1.285364,2.313208,...,2.200287,2.402425,-0.535500,-0.438531,2.464911,2.123102,2.677406,0.964723,0,0.877516
1f94bc17-7ea1-4fa2-b1a1-949f9ca1902e,2014,Savon Goodman,1.141035,0.213133,2.435173,1.667471,1.844821,2.239000,1.544299,-1.091771,2.327219,4.287415,...,3.436016,1.201299,-0.535500,-0.438531,2.464911,2.272762,2.219335,-0.383849,0,0.788372
200b9d70-e2a7-4a43-84ad-80782480645d,2014,Chris Hooper,0.545136,-0.568812,1.960033,1.298553,2.089977,2.506178,2.273295,-1.091771,0.790924,3.441326,...,2.761982,3.002989,-0.535500,-0.438531,2.464911,1.823780,2.494177,1.414247,0,0.810576
236e598c-831d-4b17-b121-5b9fe5881305,2015,Yante Maten,1.141035,0.995078,2.435173,3.880980,4.051229,0.101576,0.086309,0.170032,1.691511,0.903060,...,2.087948,-0.600392,0.783245,1.968212,0.322416,4.368012,3.868389,1.414247,0,0.830023
272b0157-c3cf-409e-80a6-9a7f7500a5b7,2014,Javonte Green,1.141035,1.777023,3.147883,2.405307,3.070603,2.506178,2.273295,-2.353574,0.843900,2.172193,...,3.098999,7.507214,-0.205813,0.363717,1.929287,3.021066,3.318704,-1.732422,0,0.781100


In [220]:
similar_players = df.query(f'cluster=={no_cluster_hachimura}')

In [198]:
df_similar = df.sort_values('cosine', ascending=False)
df_similar

,,,assists,blocks,defensive_rebounds,field_goals_att,field_goals_made,free_throws_att,free_throws_made,height,minutes,offensive_rebounds,...,rebounds,steals,three_points_att,three_points_made,turnovers,two_points_att,two_points_made,weight,cluster,cosine
player_id,season,full_name,,,,,,,,,,,,,,,,,,,,,
66a86e68-5bfc-4e7a-9610-1a7a7ae406b4,2018,Rui Hachimura,1.083755,0.495980,1.443748,2.101433,2.636474,2.455580,2.524234,0.133618,1.363374,0.387861,...,1.183667,1.035986,0.080286,0.192487,1.140384,2.553093,2.903618,0.104415,4,1.000000
5ddccd3b-97ed-4a08-92ac-77c6c1e28b8c,2015,Anthony Gill,1.369218,0.571446,1.634529,1.251621,2.318245,2.063970,2.394934,0.133618,1.181873,0.052725,...,1.183667,1.670372,-0.530110,-0.430721,0.566942,1.791214,2.758154,0.556084,4,0.975191
48a44e9f-d17d-46af-a730-b7563e63101e,2015,Perry Ellis,0.227367,-0.183211,0.837555,1.847133,2.434824,1.802897,2.040555,0.133618,1.368028,0.259392,...,0.697851,-0.142159,0.097241,0.337903,0.827597,2.231675,2.626882,0.104415,4,0.974156
2c4481c6-6c41-47a7-b9c6-f8ae3a54a185,2016,Chima Moneke,-0.058096,0.194117,2.773063,1.847133,2.434824,2.455580,2.926502,-1.122892,1.344759,1.292728,...,2.479175,1.217239,-0.530110,-0.430721,1.348908,2.525317,2.889426,-0.076253,4,0.948793
33ed07ae-50b7-435e-b3a8-4ed53b86d780,2013,Dustin Hogue,1.274064,-0.560539,0.344190,1.807432,2.901139,2.455580,2.690249,-1.122892,1.872200,1.981618,...,1.021728,1.821416,0.306358,0.081695,1.001368,2.084855,3.239485,-0.347254,4,0.947522
b40cd2a0-78bf-497a-bb4c-b4bc35c715a5,2013,James Michael McAdoo,2.225606,0.194117,1.634529,2.561748,1.968509,2.194507,1.508987,0.761873,1.391298,1.706062,...,1.831421,1.217239,-0.530110,-0.430721,0.827597,3.406240,2.364338,0.556084,4,0.940974
480f8b24-cc49-4b47-8d21-cfc4937e0d6f,2015,Martin Breunig,1.654681,-0.560539,2.317650,1.966236,2.901139,1.933433,2.217745,0.133618,1.810148,0.052725,...,1.669482,-0.595291,-0.530110,-0.430721,2.391530,2.672137,3.414514,-1.250593,4,0.938383
7963c2e6-8c7a-450d-a36f-12c9fa9a3354,2014,Montrezl Harrell,2.225606,0.948774,1.179115,1.668480,1.851930,2.325043,2.217745,0.133618,2.182459,0.879393,...,1.183667,-0.142159,-0.530110,-0.430721,0.827597,2.305086,2.233067,1.459423,4,0.938042
0af749be-c626-49f5-a558-aec6141a3421,2016,TJ Leaf,0.132213,0.445670,1.406822,1.728031,2.279386,2.107482,1.272735,1.390128,1.220655,0.603837,...,1.237646,-0.595291,0.515475,1.106527,0.653827,1.889094,2.189309,0.104415,4,0.932540


## 八村塁（Rui Hachimura）の NBA初年度成績を予測する

### 基本的なアイディア

大学時代に八村選手と同じようなスタッツを残した選手を見つけ出し、その選手が NBA 初年度に残した成績を参照することで、八村選手の活躍もある程度推測できるのではないか？

### データ

Google BigQuery に `bigquery-public-data/ncaa_basketball` という公開データがあるのでこれを使用する。
もともと kaggle で使用された課題用のデータだが、実際の NCAA のスタッツが実名で得られる。

```SQL
SELECT
    *
FROM
    `bigquery-public-data.ncaa_basketball.mbb_teams`
WHERE
    market="Gonzaga"
LIMIT 1;
```

として、チームID を取得したあとで、

```SQL
SELECT
    *
FROM
    `bigquery-public-data.ncaa_basketball.mbb_players_games_sr`
WHERE
    team_id="2f4d21f8-6d5f-48a5-abca-52a30583871a" AND
    first_name="Rui" AND
    tournament = "NCAA"
    LIMIT 1000;
```

とすると八村選手の登録されている試合の結果が取得できる。
BigQuery上で集計まで行っても良いが、できる限り python側に raw に近い状態で持ってきた方が扱いが楽なため、試合ごとのデータをダウンロードする。
なお、クエリ結果は 1GB までであればそのまま直接 json で Google Drive上に保存できる。

そして、まずは八村選手と身長、体重、ポジションの近いプレイヤーを抜き出す。

```SQL
SELECT
    *
FROM
     `bigquery-public-data.ncaa_basketball.mbb_players_games_sr`
 WHERE
     height BETWEEN 75 AND 85 AND
     weight BETWEEN 205 AND 245 AND
     position="F" AND
     season BETWEEN 2011 AND 2015 AND
     tournament = "NCAA"
LIMIT 100000;
```

BigQuery上では 2003年のデータから取得できるが、あまり昔のデータを引っ張ってきてもルールや文化が変わってしまっているので 2011年から 2015年に大学でプレーしていた選手に限定する。
また、身長は ±5-inch、体重は ±20-pounds とし、ポジションはフォワード（F）のプレイヤーだけを取得する。

ただ、BigQuery上にあるデータが 2017年までのものなので、八村選手がメインでプレーした 2018年のデータはない。
そこで、 ESPN から 2018年のデータを取得する。
https://www.espn.com/mens-college-basketball/team/stats/_/id/2250

実はこの ESPN から取得したデータは kaggle で用いられたものよりも粒度が荒く（とはいえバスケットボールで通常用いられる指標である）、このまま結合しようとすると色々と不都合があるのであとで再整形する。

### データ読み込み

BigQuery から取得したデータは jsonl形式（一行ずつが独立した jsonフォーマットになっているテキスト形式）なので、これを python で読み込んで pandas.DataFrame の形に変換する。

```python
import pandas as pd

DATA_PATH_RUI = './data/bq-results-20190710-040500-jn11labq5atn.json'
DATA_PATH_SIM = './data/bq-results-20190711-072404-txneu5c4yv68.json'


# jsonl を読み込んで pandas.DataFrame に変換する関数
def read_jsonl(path):
    with open(path, 'r') as f:
        data = f.readlines()
        
    data = map(lambda x: x.rstrip(), data)
    data_json_str = "[" + ','.join(data) + "]"
    return pd.read_json(data_json_str)


# 八村選手のデータを読み込む
df_rui = read_jsonl(DATA_PATH_RUI)
# 他の選手のデータを読み込む
df_sim = read_jsonl(DATA_PATH_SIM)
```

ちなみにこの時点で何人くらいの候補プレイヤーがいるかというと、

```python
names = df_sim.abbr_name.unique()
print(f'The number of similar players to Rui Hachimura = {len(names)}')
# => The number of similar players to Rui Hachimura = 1300
```

ということで 1,300人が該当した。
八村選手の身長は 2m を超え、体重は 100kg 近いが、5年間で同じような体格の選手がこれだけいるということである。
アメリカの大学バスケ市場の凄まじさがこれだけで伝わってくる。


### 前処理（preprocess）

さて、BigQuery から取得したデータは各試合ごとのスコア（ボックススコアと呼ばれる）なので、これを各年で集計する必要がある。
基本的にバスケットボールの世界では統計値には平均値のみを使用し、分散その他統計値はあまり用いられない。
ESPN のデータも例外ではなく平均値のみなので、平均を計算する関数で簡単に済ませられる。

さて、`DataFrame.mean()`メソッドを使用したいのだが、ここで注意しなければならない点がいくつかある。

1. 文字列は `mean()` によって無視される
2. 割合の平均は全体の割合に等しくない
3. 欠損値の扱い
4. （カラム毎のスケール）

今回使用するデータは非常に綺麗なフォーマットである。
綺麗とはどういうことかというと、文字列はちゃんと文字列として格納してあるし、数値は数値として格納してある、...ということである。
「何を当たり前のことを...？」と思う人もいるかもしれない。
それが、こと日本国内においてはこの当たり前でないデータが数多くある。

原理的に仕方ないものとしては、元号が一つの例として挙げられる。
元号は知っての通り、`令和元年` などと表記される。
この `元年` とは `1st year`　のことであり、当然 `2nd year` と連続する値である。
しかし、多くのソフトウェア、ミドルウェアでは `令和元年` と `令和2年` は連続する整数ではなく、ただの文字列として扱われる。
そこで、年次毎の売上推移を見たければ元号から西暦に変換する必要がある。

原理的に仕方無くないものに関しては、ここでは言及しない。
"ネ申エクセル" と揶揄されるものは大方これを含む。

話はそれたが、綺麗なフォーマットにおいては、`mean()` は正しく機能する。
何も考えずに `DataFrame.groupby(...).mean()` とすれば、`groupby()` した中での平均値を求めてくれる。
今回の例で言えば、`player_id`、`season`、`full_name` が固有なデータ行について集計してくれ、というものである。
多くの場合、`player_id` と `full_name` は一致するので、どちらか一方を指定すればよいのだが、あとでデータを見るときにどちらもあった方が便利なため残しておく。

ここまでで、綺麗なフォーマットであれば `mean()` で機械的に平均が計算できるとのべたが、**割合の平均は全体の割合に等しくない** という点に注意する必要がある。

例えば、一試合目で 2本打って 1本入り 50%、二試合目で 3本打って 2本入り 66.6..% とする。
このとき、割合の平均をとると、 `(50 + 66.6...) / 2 = 116.6... / 2= 58.3...` となる。
一方、全体では 5本打って 3本入ったのだから、`3 / 5 = 0.6 = 60%` である。
このように、割合の平均を取ると、全体の割合から歪んでしまうことに注意しなければならない。

最後に、欠損値について考えなければならない。
欠損値とはその名の通り、記録できていないデータである。
例えば、出場機会がなく得点が 0点のとき、そのまま 0 としてしまうとあとで困る（特に平均や除算を行う場合）ので、
NA や NaN といった疑似的な数値を入れることがある。
この欠損値をそのままにしても上手くいくアルゴリズムはいくつかある（[How to deal with Missing Value - xgboost](https://xgboost.readthedocs.io/en/latest/faq.html#how-to-deal-with-missing-value）)のだが、大抵はそうでない。
かといって、前述のようにただ 0 としてしまうのはあまり良くない。

そこで、「何もしない」、「平均値または中央値で埋める」、「最頻値で埋める」などといったヒューリスティックが使われている（[6 Different Ways to Compensate for Missing Values In a Dataset (Data Imputation with examples)](https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779)）。

しかし、これをすれば良いというものはなく、データセットや問題に応じてどの方法で欠損値を扱うかを毎回検討せねばならず、このことがデータサイエンティストが当たる最初の壁となる。

さらにもう一つ、処理するアルゴリズムによっては、カラムごとの値のスケールを調整しなければならない場合もある。
例えば、平均得点が 5点違うのと、平均ブロック数が 5つ違うのとでは、意味合いが全く異なる。
選手の類似性という点に着目すると決めている以上、この差異が十分よく反映されるように変換を施すべきである。

さて、前置きが長くなったが、今回の場合に照らすと

1. パーセンテージの平均は使わない（ちゃんと計算する）
2. 欠損値を埋める必要がある
3. 値を正規化、標準化する必要がある

となる。以下に具体的な処理を示す。

#### 使用するカラムを指定する

DataFrame を扱う際、カラムを指定して処理を行いたい場合が多いので、あらかじめ処理対象のカラムを記憶しておくと良い。
ここでは、`num_labels`という変数に数値型の値を格納している。

```python
str_labels = [
    # プレイヤーID
    'player_id',
    
    # プレイヤー名
    'full_name',

    # シーズン
    'season',    
]

num_labels = [
    # アシスト数
    'assists',

    # ブロック数
    'blocks',
    
    # ディフェンスリバウンド数
    'defensive_rebounds',

    # フィールドゴール試投数
    'field_goals_att',
    
    # フィールドゴール成功数
    'field_goals_made',
    
    # フリースロー試投数
    'free_throws_att',
    
    # フリースロー成功数
    'free_throws_made',
    
    # 身長
    'height',
    
    # 出場時間
    'minutes',
    
     # オフェンスリバウンド数
    'offensive_rebounds',
    
    # 得点数
    'points',
    
    # リバウンド数
    'rebounds',
    
    # スティール数
    'steals',
    
    # 3P試投数
    'three_points_att',
    
    # 3P成功数
    'three_points_made',
    
    # ターンオーバー数
    'turnovers',
    
    # 2P試投数
    'two_points_att',
    
    # 2P成功数
    'two_points_made',
    
    # 体重
    'weight',
]
```

#### 欠損値処理

今回は、欠損値はそのカラムの平均値で埋めるという処理を施す。

```python
# 値が数値か否かを判定するメソッド
def is_number(x):
    return (type(x) is int or type(x) is float) and not np.isnan(x)
```

```python
# 「値が数値ならそのままの値を、そうでなければ平均値を返すメソッド」を返すメソッド
def value_or_mean(df, label):
    mean = df[label].dropna().mean()

    def f(row):        
        v = None
        if is_number(row[label]):
             v = row[label]
        else:     
             v = mean
        return v
    return f


# 欠損値を埋める処理
def fillna(df):
    _df = df.copy()
    
    neighbors = []

    for label in num_labels:
        _df[label] = df.apply(value_or_mean(df, label), axis=1)
    return _df


df_sim = fillna(df_sim)
```

#### 統計処理

ここまで来て、やっと統計処理を行うことができる。

```python
# 八村選手のスタッツをシーズンごとに計算
df_rui_stats = df_rui.groupby(['player_id', 'season', 'full_name']).mean()

# 他のプレイヤーのスタッツをシーズンごとに計算
df_sim_stats = df_sim.groupby(['player_id', 'season', 'full_name']).mean()
```

#### 標準化

今回のケースでは、平均を引いて標準偏差で割る標準化を行う。
なぜ閉区間`[0, 1]` へのフラットな射影ではなく標準化をするかというと、今回重要なのは平均に比べてどのくらい異質なスコアを各プレイヤーが示しているかという点だからである。
例えば、平均得点が 30点のプレイヤーがいたとしよう。このとき、他のプレイヤーも同様に 30点近くを記録しているのであれば、そのプレイヤーは平均的な得点力、と見れるかも知れない。一方、他のプレイヤーの平均得点が高々10点近くなのであれば、そのプレイヤーの得点力は並外れていると評価できる。
このように、平均からどのくらい離れているか、異常であるかを示す変換として標準化を使用できる。

```python
def standardize(df, labels):
    _df = df.copy()
    for label in labels:
        # 平均と標準偏差を計算
        mean = df[label].mean()
        std = df[label].std()

        values = []
        for val in df[label].values:
            
            # 標準偏差が 0 でなければ標準化する
            if std > 0:
                values.append((val - mean) / std)
                
            # 標準偏差が 0 の場合は、どのプレイヤーも同じ値をとるということなので、
            # とりあえず 0 を入れておけば問題ない（厳密には平均を入れるが、類似度計算にはあまり関係ない）
            else:
                values.append(0.0)
                
        # ラベルに示されるカラムを更新
        _df[label] = values
    return _df

df = standardize(df, num_labels)
```

さて、ここまでが前処理である。
次節では、いよいよ類似度計算に移る。


### 類似度計算（Cosine類似度）

ある 2つのベクトルが類似しているとはどういうことだろうか？
例えば各ベクトルの大きさをとって、それぞれが近い値なら類似しているといえるかもしれない。
2つのベクトルを超平面上にプロットして、その距離が近くても類似しているといえるかもしれない。
さて、ここでの類似とは、「2人のプレイヤーが似たタイプのプレイを得意とする選手である」と定義する。
すなわち、似たプレイスタイルであれば、その強弱は考えない、ということである。
なぜこのように定義するかというと、NBA にドラフトされるような選手はその時点である程度プレイスタイルが完成されており、リーグに入ってもそれほど大きくプレイスタイルを変えないだろう、という知見による。
プレイスタイルが変わらないのであれば、異なるのはその洗練度であり、その選手がどう成長していくかも含めて参考になるだろう。

これらを踏まえ、類似度には Cosine類似度を採用する。
Cosine類似度は、ベクトルの内積をそれぞれの大きさの積で割ったものとして定義される。
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

幾何学的には、2つのベクトルの角度が小さいほど、値が大きくなる指標である。
Cosine類似度は自分で実装しても良いのだが、python では `scikit-learn` に実装されているため、それを使用する。
このような指標（metric）は意外とハマるので極力ありものを使ったほうが良い。
ハマりどころは境界値（0 や inf）が多い。

```python
from sklearn.metrics.pairwise import cosine_similarity

def calc_cos(row, df):
    cosines = []
    for _, x in df.iterrows():
        X = row.values.reshape(1, -1)
        Y = x.values.reshape(1, -1)
        cosines.append(cosine_similarity(X, Y)[0][0])
    return cosines

# iloc[0] は八村選手のデータ
cos = calc_cos(df.iloc[0], df)

df['cosine'] = cos
```

Cosine類似度が計算できたら、後はソートするだけである。

```python
df_similar = df.sort_values('cosine', ascending=False)
```

この結果は、公開した jupyter notebook に載せているので興味があればご覧いただきたい。


## 結果

さて、以上より、Cosine類似度において類似していると思しき選手のリストを手に入れた。
そこで、類似度の高い選手について、NBA でどんな成績を残しているか見ていく。

ここで注意すべきなのは、Cosine類似度を用いているため、類似性はあくまでプレイスタイルに関してであり、実際にはそれほど活躍していない選手も載っているということだ。
そのような選手と、八村選手のようにリーグから注目されている選手とを区別するために、ドラフトされているか否かを指標に入れる。
ただし、このデータは配布されていないため、名前を Google検索して目でドラフト順位を記載したものである。
また、ドラフト外の選手は上述の理由で取り上げないこととする。

### 1st pick of Rui-like draft, Montrezl Harrell（Houston Rockets, 32nd）

まずは最もプレイスタイルが近いと判定された選手、Montrezl Harrell。
https://www.basketball-reference.com/players/h/harremo01.html

現在は LA Clippers に所属しており、カンファレンス・ファーストラウンドで昨季の王者である Golden State Warriors を苦しめた。

初年度成績はまずまずといったところ。
全体32位での指名なので、八村選手はこれよりも良いスタッツが期待できる。

エースとまではいかないが、若いチームで安定した数字を残しており、4年目でこのくらいの数字であれば NBA に残り続けることができるだろう、という成績だ。
ちなみに契約金は 600万ドル（およそ7億円）と報じられている。
https://www.espn.com/nba/team/roster/_/name/lac/la-clippers

ただし今シーズンは大型選手補強が行われており、プレイタイムは減るかも知れない。


#### 初年度成績（Houston Rockets）

* 39ゲーム出場（1ゲーム先発）
* 平均9.7分プレイ
* 平均3.6得点
* 平均1.7リバウンド
* 平均0.3スティール
* 平均0.3ブロック
* 平均0.4ターンオーバー
* 平均1.2パーソナルファウル


#### 最新シーズン成績（4年目：LA Clippers）

* 82ゲーム出場（5ゲーム先発）
* 平均26.3分プレイ
* 平均16.6得点
* 平均6.5リバウンド
* 平均0.9スティール
* 平均1.3ブロック
* 平均1.6ターンオーバー
* 平均3.1パーソナルファウル

### 2nd pick of Rui-like draft, T.J. Leaf（Indiana Pacers, 18th）

イスラエル出身のプレイヤー T.J. Leaf。
https://www.basketball-reference.com/players/l/leaftj01.html

八村選手と同じく 1ラウンドでの指名（18位）であるが、正直なところ、それほど目立つ活躍はできていない。
趣旨が変わってしまうのであまり詳しくは調査していないが、もしかするとコミュニケーションの問題があったのかもしれない。

#### 初年度成績（Indiana Pacers）

* 53ゲーム出場（0ゲーム先発）
* 平均8.7分プレイ
* 平均2.9得点
* 平均1.5リバウンド
* 平均0.1スティール
* 平均0.1ブロック
* 平均0.2ターンオーバー
* 平均0.8パーソナルファウル


#### 最新シーズン成績（2年目：Indiana Pacers）

* 58ゲーム出場（1ゲーム先発）
* 平均9.0分プレイ
* 平均3.9得点
* 平均2.2リバウンド
* 平均0.2スティール
* 平均0.3ブロック
* 平均0.2ターンオーバー
* 平均0.6パーソナルファウル

### 3rd pick of Rui-like draft, Marquese Chriss（Sacramento Kings, 8th）

全体8位指名で、今回最も八村選手と指名順が近い Marquese Chriss選手。
https://www.basketball-reference.com/players/c/chrisma01.html

初年度はなんと全ゲームに出場、内75ゲームで先発するという期待のされ方。
また、スタッツも初年度にしては非常に良く、上々のスタートだったが、現在はトレードされて出場機会もぐっと減ってしまっている。


#### 初年度成績（Sacramento Kings）

* 82ゲーム出場（75ゲーム先発）
* 平均21.3分プレイ
* 平均9.2得点
* 平均4.2リバウンド
* 平均0.8スティール
* 平均0.9ブロック
* 平均1.3ターンオーバー
* 平均3.2パーソナルファウル

#### 最新シーズン成績（5年目：Cleveland Cavaliers）

* 27ゲーム出場（2ゲーム先発）
* 平均14.6分プレイ
* 平均5.7得点
* 平均4.2リバウンド
* 平均0.6スティール
* 平均0.3ブロック
* 平均0.9ターンオーバー
* 平均2.4パーソナルファウル


### 考察

チームの育成、起用方針に大きく左右されるので一概には言えないが、実力や期待度的には Marquese Chriss と同じくらいのプレイタイムが与えられ、成績を残してもおかしくない。
一方で、やはりそれほど甘くない世界、成績が伸び悩めばトレード、最悪解雇されることもありうる。
八村選手は渡邊選手とともに日本の期待を大きく背負っているが、直近の成績を重視せず、 Montrezl Harrell のように成長を続けて欲しい。

### 結論（予想）

Washinton Wizards の事情も考えて、だいたい Marquese Chriss の上位互換となるような成績を残すのではないだろうか。

* 70ゲーム出場（56ゲーム先発）
* 平均25.0分プレイ
* 平均14.0得点
* 平均6.0リバウンド
* 平均0.8スティール
* 平均2.0ブロック
* 平均1.3ターンオーバー
* 平均3.2パーソナルファウル